In [1]:
# %load my_imports.py
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import csv

import datetime
import time

import json



In [2]:
# Dict writer n reader
import csv
import re
def write_to_file(name, dict_object):
    with open(name, 'wb') as csv_file:
        writer = csv.writer(csv_file)
        for key, value in dict_object.items():
           writer.writerow([key, value])
        
def load_from_file(file_name):
    dict_object = {}
    df = pd.read_csv(file_name, names = ['key', 'value'])
    for idx in df.index:
        key = df.key[idx]
        value = [int(value) for value in df.value[idx][1:-1].split(',')]
        dict_object.update({key: value})
    return dict_object

def load_lists(file_name):
    dict_object = {}
    df = pd.read_csv(file_name, names = ['value', 'key'])
    for idx in df.index:
        dict_object.update({df.key[idx]: df.value[idx]})
    return dict_object

def load_lists_reversed(file_name):
    dict_object = {}
    df = pd.read_csv(file_name, names = ['key', 'value'])
    for idx in df.index:
        dict_object.update({df.key[idx]: df.value[idx]})
    return dict_object

def load_pagerank(file_name):
    dict_object = {}
    df = pd.read_csv(file_name, names = ['key', 'value'])
    for idx in df.index:
        dict_object.update({int(df.key[idx]): df.value[idx]})
    return dict_object


In [3]:
# load both index lists
movie_list = load_lists('10 networks/movie_dict.csv')
# actor_list = load_lists('10 networks/actor_dict.csv')
movie_actor_dict_int_int = load_from_file('10 networks/movie_actor_dict_int_int_trimmed.csv')
# actor_movie_dict_int_int = load_from_file('10 networks/actor_movie_dict_int_int.csv')
# actor_list_reversed = load_lists_reversed('10 networks/actor_dict.csv')
movie_list_reversed = load_lists_reversed('10 networks/movie_dict.csv')

In [4]:
# Find indexes 
target_movies = ['Batman v Superman: Dawn of Justice (2016)', 'Mission: Impossible - Rogue Nation (2015)', 'Minions (2015)']
target_movie_index = [movie_list_reversed[movie] for movie in target_movies]
target_movie_index

[267804, 415523, 289502]

In [5]:
# load pagerank score
page_rank = load_pagerank('others/pagerank.txt')
len(page_rank)

113043

In [6]:
# constuct list of movies that are directed by top 100 director
# Turned out This is not a good method as most movies are tainted
# use dict instead of df here for speed
movie_ratings = {}
with open ('movies/movie_rating.txt') as f:
    contents = f.readlines()
for content in contents:
    line = filter(None, re.split(r"[~\t\n]+", content))
    movie_ratings.update({line[0]: float(line[-1])})

In [7]:
# Construct movie rating dataframe
# try - except just simply pass key errors
movie_int_ratings = {}
no_ratings = 0
for key in movie_actor_dict_int_int.keys():
    try:
        movie_int_ratings.update({key: [movie_ratings[movie_list[key]]]})
    except: 
        no_ratings +=1
print(no_ratings)
print(len(movie_actor_dict_int_int))
print(len(movie_int_ratings))

21111
105596
84485


In [8]:
movie_int_ratings[2]

[3.8]

In [9]:
df_movie_ratings = pd.DataFrame.from_dict(movie_int_ratings).transpose()
df_movie_ratings.head()
len(df_movie_ratings)

84485

In [10]:
df_movie_ratings.columns = ['rating']
df_movie_ratings.head()

,rating
2,3.8
3,5.4
4,7.2
7,6.2
9,9.8


In [11]:
df_movie_ratings['dir_famous'] = 0
df_movie_ratings.head()

,rating,dir_famous
2,3.8,0
3,5.4,0
4,7.2,0
7,6.2,0
9,9.8,0


In [12]:
# Assigning pagerank score to the df
page_rank_score = []
for movie in df_movie_ratings.index:
    pagerank_score = [page_rank[actor] for actor in movie_actor_dict_int_int[movie]]
    pagerank_score.sort()
#     page_rank_score.append(sum(pagerank_score[:5]))
    page_rank_score.append([pagerank_score[:5]])
# df_movie_ratings['pagerank'] = page_rank_score

In [13]:
for i in range(5):
    page_rank_temp = [score[0][i] for score in page_rank_score]
    df_movie_ratings['%d' %i] = page_rank_temp
df_movie_ratings.head()

,rating,dir_famous,0,1,2,3,4
2,3.8,0,0.000005,0.000007,0.000009,0.000009,0.000010
3,5.4,0,0.000005,0.000005,0.000005,0.000005,0.000005
4,7.2,0,0.000004,0.000004,0.000005,0.000005,0.000005
7,6.2,0,0.000005,0.000006,0.000007,0.000009,0.000010
9,9.8,0,0.000003,0.000003,0.000004,0.000004,0.000006


In [14]:
# prepare test set
df_target_movie_ratings = df_movie_ratings.ix[target_movie_index]
target_rating = {267804:7.1, 415523:7.5, 289502:6.4}
for idx in target_movie_index:
    df_target_movie_ratings.set_value(idx, 'rating', target_rating[idx])
    pagerank_score = [page_rank[actor] for actor in movie_actor_dict_int_int[idx]]
    pagerank_score.sort()
    for i in range(5):
        df_target_movie_ratings.set_value(idx, '%d' %i, pagerank_score[i])
    df_target_movie_ratings.set_value(idx, 'dir_famous', 0)
df_target_movie_ratings

,rating,dir_famous,0,1,2,3,4
267804,7.1,0.0,0.000004,0.000004,0.000005,0.000005,0.000005
415523,7.5,0.0,0.000005,0.000005,0.000006,0.000006,0.000006
289502,6.4,0.0,0.000003,0.000006,0.000006,0.000010,0.000016


In [15]:
# director_dictionary
import re
with open ('movies/director_movies.txt') as f:
    contents = f.readlines()
director_dict = {}
for content in contents:
    line = filter(None, re.split(r"[~\t\n]+", content))
    name = line[0].split('(')[0].rstrip()
    director_dict.update({name: line[1:]})

In [16]:
filter(None, re.split(r"[~\t\n]+", contents[6]))[0].split('(')

["'Kusare, Mak ", 'I)']

In [17]:
# Instead we simply scrapy from the page
from urllib import urlopen
 
from bs4 import BeautifulSoup
html = urlopen("http://www.imdb.com/chart/top")
html_soup = BeautifulSoup(html, 'html.parser')
tbody = html_soup.find('tbody', {'class': 'lister-list'})
names = tbody.findAll('td', {'class': 'titleColumn'})
ratings = tbody.findAll('td', {'class': 'ratingColumn imdbRating'})

In [18]:
# Construct a top-250 list from here
import shlex
movie_rating_top_250 = pd.DataFrame()
for name, rating in zip(names, ratings):
    movie_name = re.split(r"[~<>\n]+", str(name))[5] + ' ' + re.split(r"[~<>\n]+", str(name))[8]
    movie_rating = re.split(r"[~<>\n]+", str(rating))[3]
    # This is by trail and error easy one-liner
    movie_director = str.rstrip(re.split(r"[~=()]+",shlex.split(re.split(r"[~<>\n]+", str(name))[4])[2].split(',')[0])[1])
    movie_rating_top_250 = movie_rating_top_250.append([[movie_name, movie_rating, movie_director]])
movie_rating_top_250.reset_index(inplace=True, drop=True)
movie_rating_top_250.head()

,0,1,2
0,The Shawshank Redemption (1994),9.2,Frank Darabont
1,The Godfather (1972),9.2,Francis Ford Coppola
2,The Godfather: Part II (1974),9.0,Francis Ford Coppola
3,The Dark Knight (2008),8.9,Christopher Nolan
4,12 Angry Men (1957),8.9,Sidney Lumet


In [19]:
# Instead of creating another director list,
# what we can do is do a counting till we have 100 directors,
# and tags all movies examined so far.
def movies_by_100_dir(seq):
    keys = {}
    count = 0
    for e in seq:
        count += 1
        keys[e] = 1
        if len(keys) == 100:
            break
    return count, keys.keys()
counts, directors = movies_by_100_dir(movie_rating_top_250[2])

In [20]:
# Off 166 movies by 100 dir,
# 38 of them are not on the list
df_movie_ratings['dir_famous'].value_counts()
df_movie_ratings.to_csv('ratings/ratings_raw.csv', header = 1, index = 1)

# df_movie_ratings = pd.read_csv('ratings/ratings_raw.csv')
# df_movie_ratings = df_movie_ratings.set_index("Unnamed: 0")
# del df_movie_ratings.index.name
# df_movie_ratings.head(3)

In [21]:
# map directors into 1 column
movie_counts = 0
director_counts = 0
for director in directors:
    rename = director.split(' ')
    name = rename[1] + ', ' + rename[0]
    if name in director_dict.keys():
        director_counts = director_counts + 1
        for movie in director_dict[name]:
            if movie in movie_list_reversed:
                if movie_list_reversed[movie] in df_movie_ratings.index:
                    df_movie_ratings.set_value(movie_list_reversed[movie], 'dir_famous', 1)
                    movie_counts = movie_counts + 1

print ("movie counts: %d" %movie_counts)
print ("director counts: %d" %director_counts)

movie counts: 1052
director counts: 90


In [22]:
df_movie_ratings.to_csv('ratings/movie_rating_single.csv', header = 1, index = 1)

In [23]:
# define fitting functions
import sklearn.model_selection as sms
from sklearn.svm import SVR
from sklearn.linear_model import SGDRegressor
import matplotlib.pyplot as plt

def fit_sgd(df_fit, df_predict, features):
    X = df_fit[features]
    y = df_fit['rating']
    sgd_l2 = SGDRegressor(loss='huber', penalty='l2')
    X_test = df_predict[features]
    y_real = df_predict['rating']
    print('fitting started')
    start_time = time.time()
    clf = sgd_l2.fit(X, y)
    print('fitting complete. Time used: %d' %(time.time() - start_time))
    y_test = sgd_l2.predict(X_test)
    print ('cross validation: %f' %clf.score(X_test, y_test))
    return y_test, y_real

In [24]:
import datetime
import time
from sklearn.model_selection import learning_curve
from sklearn import linear_model as lm
from sklearn.metrics import mean_squared_error
from math import sqrt

def plot_learning_curve(estimator, title, file_dir, X, y, ylim=None, cv=None,
                        train_sizes=np.linspace(.1, 1.0, 5)):
    plt.figure()
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=5, n_jobs=1, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.xlabel("Training examples")
    plt.ylabel("Score")
    plt.legend(loc="best")
    plt.grid("on")
    if ylim:
        plt.ylim(ylim)
    plt.title(title)
    plt.savefig(file_dir, dpi = 800)

In [25]:
# Learning Curve
features = ['0', '1', '2', '3', '4']
sgd = SGDRegressor(loss='huber', penalty='l2')
X = df_movie_ratings[features]
y = df_movie_ratings['rating']
plt.savefig('pg01.jpg')
plot_learning_curve(sgd, 'Page Rank Score Only', 'pg01.jpg', X, y)
# Cross Validation
train, test = sms.train_test_split(df_movie_ratings, test_size=0.4, random_state=int(time.time()))
y_pred, y_real = fit_sgd(train.sort_index(), test.sort_index(), features=features)
print ('mse: %f' %mean_squared_error(y_real, y_pred))
print ('aver: %f' %np.mean(y_pred - y_real))
print ('armse: %f' %sqrt(mean_squared_error(y_real, y_pred)))

fitting started
fitting complete. Time used: 0
cross validation: 1.000000
mse: 1.414614
aver: 0.109989
armse: 1.189376


In [65]:
# Actual Rating Model 1
y_pred, y_real = fit_sgd(train.sort_index(), df_target_movie_ratings.sort_index(), features=features)
y_pred

fitting started
fitting complete. Time used: 0
cross validation: 1.000000


array([ 6.15722109,  6.15722109,  6.15722109])

In [68]:
# Learning Curve
features = ['0', '1', '2', '3', '4', 'dir_famous']
sgd = SGDRegressor(loss='huber', penalty='l2')
X = df_movie_ratings[features]
y = df_movie_ratings['rating']
plt.savefig('pg01.jpg')
plot_learning_curve(sgd, 'Page Rank Score Only', 'pg02.jpg', X, y)
# Cross Validation
train, test = sms.train_test_split(df_movie_ratings, test_size=0.4, random_state=int(time.time()))
y_pred, y_real = fit_sgd(train.sort_index(), test.sort_index(), features=features)
print ('mse: %f' %mean_squared_error(y_real, y_pred))
print ('aver: %f' %np.mean(y_pred - y_real))
print ('armse: %f' %sqrt(mean_squared_error(y_real, y_pred)))

fitting started
fitting complete. Time used: 0
cross validation: 1.000000
mse: 1.427424
aver: 0.120511
armse: 1.194749


In [70]:
# Actual Rating Model 2
y_pred, y_real = fit_sgd(train.sort_index(), df_target_movie_ratings.sort_index(), features=features)
y_pred

fitting started
fitting complete. Time used: 0
cross validation: 1.000000


array([ 6.17290303,  6.17290303,  6.17290303])

In [71]:
target_dummy = pd.get_dummies(df_target_movie_ratings.dir_famous)
target_dummy.columns = ['dir_not_famous']
movie_dummy = pd.get_dummies(df_movie_ratings.dir_famous)
movie_dummy.columns = ['dir_not_famous', 'dir_famous']
df_target_movie_ratings = pd.concat([df_target_movie_ratings, target_dummy], axis=1)
df_movie_ratings = pd.concat([df_movie_ratings, movie_dummy.dir_not_famous], axis=1)

In [75]:
# Learning Curve
features = ['0', '1', '2', '3', '4', 'dir_famous', 'dir_not_famous']
sgd = SGDRegressor(loss='huber', penalty='l2')
X = df_movie_ratings[features]
y = df_movie_ratings['rating']
plt.savefig('pg01.jpg')
plot_learning_curve(sgd, 'Page Rank Score Only', 'pg02.jpg', X, y)
# Cross Validation
train, test = sms.train_test_split(df_movie_ratings, test_size=0.4, random_state=int(time.time()))
y_pred, y_real = fit_sgd(train.sort_index(), test.sort_index(), features=features)
print ('mse: %f' %mean_squared_error(y_real, y_pred))
print ('aver: %f' %np.mean(y_pred - y_real))
print ('armse: %f' %sqrt(mean_squared_error(y_real, y_pred)))

fitting started
fitting complete. Time used: 0
cross validation: 1.000000
mse: 1.595977
aver: 0.081992
armse: 1.263320


In [74]:
# Actual Rating Model 3
y_pred, y_real = fit_sgd(train.sort_index(), df_target_movie_ratings.sort_index(), features=features)
y_pred

fitting started
fitting complete. Time used: 0
cross validation: 1.000000


array([ 6.15751179,  6.15751179,  6.15751179])

In [80]:
# map directors into 100 column
movie_counts = 0
director_counts = 0
features = []
for director in directors:
    rename = director.split(' ')
    name = rename[1] + ', ' + rename[0]
    if name in director_dict.keys():
        director_counts = director_counts + 1
        for movie in director_dict[name]:
            if movie in movie_list_reversed:
                if movie_list_reversed[movie] in df_movie_ratings.index:
                    df_movie_ratings.set_value(movie_list_reversed[movie], name, 1)
                    movie_counts = movie_counts + 1
                    features.append(name)
features = list(set(features))
df_movie_ratings.fillna(0, inplace=1)
print ("movie counts: %d" %movie_counts)
print ("director counts: %d" %director_counts)

movie counts: 1052
director counts: 90


In [81]:
for director in features:
    df_target_movie_ratings[director] = 0

In [82]:
#93 features
features.extend(['0', '1', '2', '3', '4'])
sgd = SGDRegressor(loss='huber', penalty='l2')
X = df_movie_ratings[features]
y = df_movie_ratings['rating']
plt.savefig('pg01.jpg')
plot_learning_curve(sgd, 'Page Rank Score Only', 'pg02.jpg', X, y)
# Cross Validation
train, test = sms.train_test_split(df_movie_ratings, test_size=0.4, random_state=int(time.time()))
y_pred, y_real = fit_sgd(train.sort_index(), test.sort_index(), features=features)
print ('mse: %f' %mean_squared_error(y_real, y_pred))
print ('aver: %f' %np.mean(y_pred - y_real))
print ('armse: %f' %sqrt(mean_squared_error(y_real, y_pred)))

fitting started
fitting complete. Time used: 0
cross validation: 1.000000
mse: 1.412056
aver: 0.114583
armse: 1.188299


In [83]:
# Actual Rating
y_pred, y_real = fit_sgd(train.sort_index(), df_target_movie_ratings.sort_index(), features=features)
y_pred

fitting started
fitting complete. Time used: 0
cross validation: 1.000000


array([ 6.16009448,  6.16009448,  6.16009448])